In [1]:
import pandas as pd
import numpy as np

In [3]:
#Read Yelp csv file
path='D:/Adarsh_Laptop/studetmater/python/pycon-2016-tutorial/data/yelp.csv'
data = pd.read_csv(path)

In [4]:
#select only rating 1 and 5 from stars columns of yelp.csv data
data = data[(data.stars == 5) | (data.stars == 1)] 


In [5]:
#Retreive only text and stars columns
X_data = data['text']
y_data = data['stars']

In [6]:
#checking for missing data and found none
X_data.isnull().sum()
y_data.isnull().sum()

0

In [7]:
X_data.shape

(4086L,)

In [8]:
y_data.shape

(4086L,)

In [9]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X_data,y_data,test_size =0.3,random_state=42)
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(2860L,)
(1226L,)
(2860L,)
(1226L,)


C:\Program Files\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
#CountVectorizer shall be used to work on text data
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm

<2860x16218 sparse matrix of type '<type 'numpy.int64'>'
	with 220980 stored elements in Compressed Sparse Row format>

In [11]:
#Vectorize Test data
X_test_dtm = vect.transform(X_test)

In [12]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y_train=lb.fit_transform(y_train)
y_test=lb.fit_transform(y_test)

In [13]:
# Applying Naive Bayes classifier 


In [14]:
#MultinomialNB for prediction
from sklearn.naive_bayes import MultinomialNB


In [15]:
#parameter  selection and tunning
vect_cv = CountVectorizer()
cv_X_data = vect_cv.fit_transform(X_data)
cv_y_data = lb.fit_transform(y_data)
from sklearn.cross_validation import cross_val_score
alpha = np.arange(0,1,0.1)
acc_score=[]
for alpha in alpha:
    nb = MultinomialNB(alpha=alpha)
    cv_accuracy = cross_val_score(nb,cv_X_data,cv_y_data,cv=10,scoring = 'accuracy').mean()
    acc_score.append(cv_accuracy)
    

C:\Program Files\Anaconda2\lib\site-packages\sklearn\naive_bayes.py:699: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc) -


In [25]:
print max(acc_score)

0.926105581903


In [26]:
 zip(np.arange(0,1,0.1),acc_score)

[(0.0, 0.41409272719498513),
 (0.10000000000000001, 0.92194130050451084),
 (0.20000000000000001, 0.92120660270379806),
 (0.30000000000000004, 0.92316859437526788),
 (0.40000000000000002, 0.92439108826279826),
 (0.5, 0.92610558190343895),
 (0.60000000000000009, 0.92414478581061366),
 (0.70000000000000007, 0.92487648435800318),
 (0.80000000000000004, 0.92438328313669893),
 (0.90000000000000002, 0.92584907727831656)]

In [27]:
#from cross_val_score , alpha=0.5 will give the best accuracy 
nb = MultinomialNB(alpha=0.5)
nb.fit(X_train_dtm,y_train)
nb_predict = nb.predict(X_test_dtm)

In [28]:
from sklearn import metrics
f1_Score_NB=metrics.f1_score(y_test,nb_predict)
print f1_Score_NB

0.949575636545


In [29]:
conf_matrix= metrics.confusion_matrix(y_test,nb_predict)

In [30]:
pd.crosstab(y_test, nb_predict, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,174,59,233
1,42,951,993
All,216,1010,1226


In [31]:
y_test[:5]

array([0, 1, 1, 1, 1], dtype=int64)

In [32]:
lb.inverse_transform(y_test)[:5]

array([1, 5, 5, 5, 5], dtype=int64)

In [33]:
TN=conf_matrix[0,0]
FP=conf_matrix[0,1]
FN=conf_matrix[1,0]
TP=conf_matrix[1,1]

In [34]:
#Classification Accuracy
Classification_Accuracy_NB=metrics.accuracy_score(y_test,nb_predict)
Classification_Accuracy_NB

0.91761827079934744

In [35]:
#Classification Error
Classification_Error_NB=1 - metrics.accuracy_score(y_test,nb_predict)
Classification_Error_NB

0.082381729200652565

In [36]:
#Sensitivity : is the classifier to detecting positive instances , also True Positive Rate" or "Recall"
recall_NB= metrics.recall_score(y_test,nb_predict)
recall_NB

0.95770392749244715

In [37]:
#Specificity: When the actual value is negative, how often is the prediction correct? 
#How "specific" (or "selective") is the classifier in predicting positive instances?

Specificity_NB=(TN / float(TN + FP))
Specificity_NB

0.74678111587982832

In [38]:
#calculate predicted probability 
##I want probability of getting rating 5 so 
nb_predict_prob=nb.predict_proba(X_test_dtm)[:,1]

In [39]:
#False Positive Rate: When the actual value is negative, how often is the prediction incorrect?
print(FP/ float(FP+TN))


0.25321888412


In [40]:
#Precision: When a positive value is predicted, how often is the prediction correct?
#How "precise" is the classifier when predicting positive instances?
print (TP/float(TP+FP))
Precision_NB= metrics.precision_score(y_test,nb_predict)

0.941584158416


In [41]:
#print rating 1 reviews which are falsely callisified as positive i.e rating 5 ,diplay False Postive review comments
print len(X_test[(nb_predict == 1) & (y_test == 0)])

#X_test[nb_predict > y_test]

59


In [42]:
#False postive review
#X_test[1781]

In [43]:
#print rating 5 reviews which are falsely callisified as negative i.e rating 1,diplay True Negative review comments
print len(X_test[(nb_predict == 0) & (y_test == 1)])
#X_test[ nb_predict < y_test]

42


In [44]:
# True Negative , review which is falsely callasified as Rating 1
#X_test[1438]

In [45]:
#First 5 probabilty of getting 5 rating
nb_predict_prob[:5]

array([ 0.99999946,  1.        ,  1.        ,  1.        ,  0.9999995 ])

In [46]:
nb_auc= metrics.roc_auc_score(y_test,nb_predict_prob)
nb_auc

0.93640677878194567

In [47]:
val= [f1_Score_NB,Classification_Accuracy_NB,Classification_Error_NB,recall_NB,Specificity_NB,Precision_NB,nb_auc]
name=['f1_score','Classification_Accuracy','Classification_Error','Recall','Specificity','Precision','AUC']

NB_DataFrame = pd.DataFrame(zip(name,val),columns=['NB_Prop','Value'])
NB_DataFrame

,NB_Prop,Value
0,f1_score,0.949576
1,Classification_Accuracy,0.917618
2,Classification_Error,0.082382
3,Recall,0.957704
4,Specificity,0.746781
5,Precision,0.941584
6,AUC,0.936407


In [48]:
#Solving this problem through Logistice regression
# We will try out all classifier and compare them all how fare to each other.

In [49]:
from sklearn.linear_model import LogisticRegression

In [50]:
#Parameter selection and tunning

from sklearn.cross_validation import cross_val_score
alpha = np.arange(1,5,0.1)
acc_score=[]
for alpha in alpha:
    lr = LogisticRegression(C=alpha)
    cv_accuracy = cross_val_score(lr,cv_X_data,cv_y_data,cv=10,scoring = 'accuracy').mean()
    acc_score.append(cv_accuracy)

In [51]:
print max(acc_score)

0.938823116849


In [80]:
#using GridsearchCV for parameter selection and tuning.
from sklearn.grid_search import GridSearchCV
grid = GridSearchCV(LogisticRegression(),param_grid={'C':np.arange(1,10,0.2),'penalty':['l1','l2']},cv=10)
grid.fit(X_train_dtm,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': array([ 1. ,  1.2,  1.4,  1.6,  1.8,  2. ,  2.2,  2.4,  2.6,  2.8,  3. ,
        3.2,  3.4,  3.6,  3.8,  4. ,  4.2,  4.4,  4.6,  4.8,  5. ,  5.2,
        5.4,  5.6,  5.8,  6. ,  6.2,  6.4,  6.6,  6.8,  7. ,  7.2,  7.4,
        7.6,  7.8,  8. ,  8.2,  8.4,  8.6,  8.8,  9. ,  9.2,  9.4,  9.6,
        9.8])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [81]:
grid.predict(X_test_dtm)

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [82]:
grid.best_params_

{'C': 4.1999999999999993, 'penalty': 'l2'}

In [83]:
# zip(np.arange(1,5,0.1),acc_score)

In [85]:
lr = LogisticRegression(C=4.19,penalty='l2')
lr.fit(X_train_dtm,y_train)

LogisticRegression(C=4.19, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [86]:
lr_pred = lr.predict(X_test_dtm)

In [87]:
f1_score_lr=metrics.f1_score(y_test,lr_pred)
f1_score_lr

0.95587506197322758

In [88]:
#Confusion Matrix
lr_conf_matrix = metrics.confusion_matrix(y_test,lr_pred) 
#print the confusion matrix for better understanding with row and columns
pd.crosstab(y_test,lr_pred,rownames=['True'],colnames=['Predicted'],margins=True)

Predicted,0,1,All
True,,,
0,173,60,233
1,29,964,993
All,202,1024,1226


In [65]:
#Classification Accuracy
Classification_Accuracy_lr=metrics.accuracy_score(y_test,lr_pred)
Classification_Accuracy_lr

0.92740619902120713

In [66]:
#Classification Error
Classification_Error_lr= 1 - metrics.accuracy_score(y_test,lr_pred)
Classification_Error_lr

0.072593800978792866

In [67]:
#Sensitivity : is the classifier to detecting positive instances , also True Positive Rate" or "Recall"
recall_lr= metrics.recall_score(y_test,lr_pred)
recall_lr

0.97079556898288011

In [68]:
TN=lr_conf_matrix[0,0]
FP=lr_conf_matrix[0,1]
FN=lr_conf_matrix[1,0]
TP=lr_conf_matrix[1,1]

In [69]:
#Specificity: When the actual value is negative, how often is the prediction correct? 
#How "specific" (or "selective") is the classifier in predicting positive instances?

Specificity_lr=(TN / float(TN + FP))
Specificity_lr

0.74248927038626611

In [70]:
#False Positive Rate: When the actual value is negative, how often is the prediction incorrect?
print(FP/ float(FP+TN))

0.257510729614


In [71]:
#Precision: When a positive value is predicted, how often is the prediction correct?
#How "precise" is the classifier when predicting positive instances?
print (TP/float(TP+FP))
precision_lr=metrics.precision_score(y_test, lr_pred)
precision_lr

0.94140625


0.94140625

In [72]:
#calculate predicted probability 
##I want probability of getting rating 5 so 
lr_predict_prob=lr.predict_proba(X_test_dtm)[:,1]

In [73]:
#False positive , review which are falsely predicted as rating 5
X_test[lr_pred > y_test].count()

60

In [74]:
#True Negative , review which are falsely predicted as rating 1
X_test[lr_pred < y_test].count()

29

In [75]:
#False Positive Rate: When the actual value is negative, how often is the prediction incorrect?
print(FP / float(TN + FP))

0.257510729614


In [76]:
#Calculate AUC for Logistice regression ,
auc_logistic = metrics.roc_auc_score(y_test,lr_predict_prob)
auc_logistic

0.96203034978756874

In [77]:
val= [f1_score_lr,Classification_Accuracy_lr,Classification_Error_lr,recall_lr,Specificity_lr,precision_lr,auc_logistic]
name=['f1_score','Classification_Accuracy','Classification_Error','Recall','Specificity','Precision','AUC']

LR_DataFrame = pd.DataFrame(zip(name,val),columns=['LR_Prop','Value'])
LR_DataFrame

,LR_Prop,Value
0,f1_score,0.955875
1,Classification_Accuracy,0.927406
2,Classification_Error,0.072594
3,Recall,0.970796
4,Specificity,0.742489
5,Precision,0.941406
6,AUC,0.962030


In [78]:

LR_DataFrame

,LR_Prop,Value
0,f1_score,0.955875
1,Classification_Accuracy,0.927406
2,Classification_Error,0.072594
3,Recall,0.970796
4,Specificity,0.742489
5,Precision,0.941406
6,AUC,0.962030


In [79]:
NB_DataFrame

,NB_Prop,Value
0,f1_score,0.949576
1,Classification_Accuracy,0.917618
2,Classification_Error,0.082382
3,Recall,0.957704
4,Specificity,0.746781
5,Precision,0.941584
6,AUC,0.936407


In [73]:
#Let List out all the accuracy and validating facts from both the algorithms
#First we will go with Naive Bayes 

In [74]:
#SVM classifier with kernel

In [89]:
from sklearn import svm
svc = svm.SVC(gamma=2,random_state=np.random.RandomState(0))
svc 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=2, kernel='rbf',
  max_iter=-1, probability=False,
  random_state=<mtrand.RandomState object at 0x000000000C26CEE8>,
  shrinking=True, tol=0.001, verbose=False)

In [90]:
svc.fit(X_train_dtm,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=2, kernel='rbf',
  max_iter=-1, probability=False,
  random_state=<mtrand.RandomState object at 0x000000000C26CEE8>,
  shrinking=True, tol=0.001, verbose=False)

In [91]:
svc_predict=svc.predict(X_test_dtm)
svc_predict

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [92]:
#accuracy
svc_acuracy=metrics.accuracy_score(y_test,svc_predict)
svc_acuracy

0.80995106035889075

In [93]:
#confusion metrix
svc_confus = metrics.confusion_matrix(y_test,svc_predict)
pd.crosstab(y_test,svc_predict,rownames=['True'],colnames=['Predict'],margins=True)

Predict,1,All
True,,
0,233,233
1,993,993
All,1226,1226


In [94]:
#recall score 
svc_recall=metrics.recall_score(y_test,svc_predict)
svc_recall

1.0

In [95]:
TN=svc_confus[0,0]
FP=svc_confus[0,1]
FN=svc_confus[1,0]
TP=svc_confus[1,1]

In [96]:
#Specificity: When the actual value is negative, how often is the prediction correct? 
#How "specific" (or "selective") is the classifier in predicting positive instances?

Specificity_svc=(TN / float(TN + FP))
Specificity_svc

0.0

In [97]:
#classifcation report
print (metrics.classification_report(y_test,svc_predict))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       233
          1       0.81      1.00      0.89       993

avg / total       0.66      0.81      0.72      1226



C:\Program Files\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [101]:
match=(y_test==svc_predict)

In [103]:
len(match)

1226

In [105]:
match.sum()/float(len(match))

0.91517128874388254